# Projeto 1 - Ciência dos Dados

Nome:Lorran Caetano Machado Lopes

Nome: Nívea de Abreu Dantas Lima

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [43]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [44]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lorra\OneDrive\Área de Trabalho\Ciencia dos Dados\P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [45]:
filename = 'cocacola.xlsx'

In [46]:
train = pd.read_excel(filename)
train.head(5)




,Treinamento,Relevância
0,@kchorrolooko @cocacola_br essa é a melhor bat...,0
1,@cocacola_br @_evertonalmeida #coquinhagelada ...,1
2,@juliettefreirec @bbb @joaoluizpedrosa @cocaco...,0
3,#nowplaying \n[ nicki minaj - super bass ]\...,1
4,#nowplaying \n[ nick jonas - spaceman ]\nen...,1


In [47]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,Relevâcia
0,rt @sysa86: @bbb @cocacola_br o bbb21 virou um...,0
1,@cocacola_br coca cola te amo meu refrigerante...,1
2,"@karolinemourab tem vezes que a vontade bate, ...",1
3,@kikiilda sede de coquinha? clica aqui 👉 https...,1
4,@sysa86 @bbb @cocacola_br exatamente e eu amo ...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto em questão é a Coca-cola, e consideramos como relevantes os tweets que continham propagandas com promoções, marcações das pessoas que falavam tanto bem quanto teciam críticas ao produto. Além disso, consideramos como irrelevanes tweets que marcavam o produto de forma aleatória sem transmitir uma postagem coerente a menção. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Abaixo definimos uma função de limpeza para remover os caracteres: enter, :, ", ', (, ), etc e manter os emojis, além de corrigir os espaçamentos. 

In [48]:
import re 
from nltk.tokenize.casual import TweetTokenizer #importando....


def cleanup(text):
    """
        Função de limpeza
        
    """
    #removeu o @ e o _, vamos tentar os links
    punctuation = '[\[\]@!-_.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(' +', ' ', text_subbed) #essa
    text_subbed = text_subbed.lower()
    text_subbed = re.sub(r'http\S+', '', text_subbed)
    t = TweetTokenizer()
    text_subbed = t.tokenize(text_subbed) #remover emojissss
    
    return text_subbed

In [49]:
cleanup("nada nada🤤🥰🥰")


['nada', 'nada', '🤤', '🥰', '🥰']

In [50]:
i = train.Treinamento[0]
tweet_limpo = cleanup(i)

In [51]:
#Percorrendo os tweets para aplicar a função de limpeza, seprarndo em listas de palavras
relevante = []
irrelevante = []

j = 0
for i in train.Treinamento:
    tweetl = cleanup(i)
    if train.Relevância[j] == 1:
        # Guardando as palavras dos tweets relevantes como lista 
        relevante += tweetl
    else:
        # Guardando as palavras dos tweets irrelevantes como lista 
        irrelevante += tweetl
        
    j += 1


irrelevante

['kchorrolooko',
 'cocacolabr',
 'essa',
 'é',
 'a',
 'melhor',
 'batata',
 'frita',
 'que',
 'existe',
 'juliettefreirec',
 'bbb',
 'joaoluizpedrosa',
 'cocacolabr',
 'não',
 'seguimos',
 'cobras',
 'serpentes',
 'pessoas',
 'tóxicas',
 'como',
 'essa',
 'euliette',
 'juliaacc',
 'a',
 'cara',
 'do',
 'glamour',
 'arrasaram',
 'muito',
 '😎',
 'erikapimenteler',
 'bbb',
 'cocacolabr',
 'mds',
 'gente',
 'para',
 'de',
 'desprezar',
 'o',
 'povo',
 'coisa',
 'feia',
 'se',
 'a',
 'juliete',
 'tivesse',
 'aqui',
 'fora',
 'ela',
 'não',
 'faria',
 'isso',
 'bbb',
 'cocacolabr',
 'as',
 'conversas',
 'da',
 'juliette',
 'estavam',
 'mais',
 'interessantes',
 'cocacolabr',
 'manda',
 'o',
 'mimooo',
 'brenucrf',
 'cocacolabr',
 'haja',
 'dinheiro',
 'pra',
 'fazer',
 'um',
 'igual',
 'hahagahaahha',
 'bbb',
 'cocacolabr',
 'dps',
 'da',
 'briga',
 'do',
 'gil',
 'a',
 'juliette',
 'com',
 'os',
 'melhores',
 'amigos',
 'se',
 'divertindo',
 'arthur',
 'e',
 'carla',
 'avisa',
 'que',
 'é',

Criando um conjunto com todas as palavras dos tweets relevantes e irrelevantes:

In [52]:
# Guardando as palavras relevantes em um pd.Series  
serie_relevante = pd.Series(relevante)
# Guardando as palavras irrelevantes em um pd.Series  
serie_irrelevante = pd.Series(irrelevante)
print(serie_relevante, serie_irrelevante)



0           cocacolabr
1       evertonalmeida
2       coquinhagelada
3                 amor
4                lindo
             ...      
5020              mais
5021              nada
5022                 🤤
5023                 🥰
5024                 🥰
Length: 5025, dtype: object 0       kchorrolooko
1         cocacolabr
2               essa
3                  é
4                  a
            ...     
1907     mcdonaldsbr
1908      cocacolabr
1909             ele
1910               é
1911            feio
Length: 1912, dtype: object


#### Frequências absolutas

Palavras em um texto são variáveis **qualitativas nominais**, portanto usaremos `value_counts()` para obter a tabela de frequências relativas e absolutas:

In [53]:
# Como serie_casmurro é uma pd.Series, podemos usar value_counts() nessa variável
tabela_relevante = serie_relevante.value_counts()
tabela_relevante.head(15)

👉           132
uma         112
sem         111
açúcar      109
de          100
que         100
a            96
com          96
coquinha     89
e            86
pizza        68
música       62
😋            56
aqui         55
cocacola     55
dtype: int64

In [54]:
tabela_irrelevante = serie_irrelevante.value_counts()
tabela_irrelevante.head(15)  

cocacolabr    144
a              37
de             37
que            32
não            32
e              32
pra            31
se             28
é              27
bbb            27
eu             19
aqui           18
com            17
na             17
manda          17
dtype: int64

#### Frequências relativas

In [55]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa.head(10)

👉           0.026269
uma         0.022289
sem         0.022090
açúcar      0.021692
de          0.019900
que         0.019900
a           0.019104
com         0.019104
coquinha    0.017711
e           0.017114
dtype: float64

In [56]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa

cocacolabr       0.075314
a                0.019351
de               0.019351
que              0.016736
não              0.016736
                   ...   
esquisito        0.000523
aliferraz        0.000523
dominosbrasil    0.000523
quarteto         0.000523
esqueça          0.000523
Length: 770, dtype: float64

In [57]:
total = relevante + irrelevante
todas_palavras = pd.Series(total)
todas_palavras



todas_palavras_relativa = todas_palavras.value_counts(True)
todas_palavras_relativa
todas_palavras_relativa['coquinha']

0.01398298976502811

### Tweet a classificar:

In [102]:
tw = "odeio"

In [103]:
tw = cleanup(tw.lower())

In [104]:
#CALCULANDO OS VALORES DOS PRIORS P(R) E P(I)
total = len (relevante)+len(irrelevante) 
P_R = len(relevante)/total
P_I = len(irrelevante)/total

print(P_R, P_I)




0.7243765316419202 0.2756234683580799


In [108]:
probTweetDadoR = 1
probTweetDadoI = 1

for i in tw:
#     probTweetDadoR *= tabela_relevante[i]+1/len(tw)
#     probTweetDadoI *= tabela_irrelevante[i]+1/len(tw)
    if i in tabela_relevante:
        probTweetDadoR *= tabela_relevante[i]
    elif i in tabela_irrelevante:    
        probTweetDadoI *= tabela_irrelevante[i]
    
print(probTweetDadoR, probTweetDadoI)

1 1


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**